In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from PIL import Image
import glob
import cv2  # OpenCV for efficient image loading
from scipy.spatial.distance import cdist
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score

In [ ]:
# First we will start by loading the data
# Load the data
df= pd.read_csv('/content/drive/MyDrive/DM Project/MetroPT3(AirCompressor).csv')
df.head()


,Unnamed: 0,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
0,0,2020-02-01 00:00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
1,10,2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2,20,2020-02-01 00:00:19,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
3,30,2020-02-01 00:00:29,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
4,40,2020-02-01 00:00:39,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [ ]:
# Drop the first column
df = df.drop(df.columns[0], axis=1)
df2=df
df.head()

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
0,2020-02-01 00:00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
1,2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2,2020-02-01 00:00:19,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
3,2020-02-01 00:00:29,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
4,2020-02-01 00:00:39,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [ ]:
df2=df

In [ ]:
df2.head()

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
0,2020-02-01 00:00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
1,2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2,2020-02-01 00:00:19,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
3,2020-02-01 00:00:29,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
4,2020-02-01 00:00:39,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [ ]:
# Create a new column 'AirLeak' and initialize it with 0
df['Airleak'] = 0

In [ ]:
# Define the air leak instances and corresponding time ranges
air_leak_instances = [
    ('4/18/2020 0:00', '4/18/2020 23:59'),
    ('5/29/2020 23:30', '5/30/2020 6:00'),
    ('6/5/2020 10:00', '6/7/2020 14:30'),
    ('7/15/2020 14:30', '7/15/2020 19:00')
]

In [ ]:
# Convert the time columns to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
# Iterate through air leak instances and set 'AirLeak' to 1 for the corresponding time range
for start_time, end_time in air_leak_instances:
    mask = (df['timestamp'] >= start_time) & (df['timestamp'] <= end_time)
    df.loc[mask, 'Airleak'] = 1

# Print the DataFrame with the new 'AirLeak' column
df.head()

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,Airleak
0,2020-02-01 00:00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
1,2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
2,2020-02-01 00:00:19,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
3,2020-02-01 00:00:29,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
4,2020-02-01 00:00:39,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0


In [ ]:
#file_path = '/content/drive/MyDrive/DM Project/labelled_df.csv'

# Save the DataFrame to a CSV file
#df.to_csv(file_path, index=False)

In [ ]:
## drop timestamps
df= df.drop('timestamp', axis=1)


df.head()


In [ ]:
df

In [ ]:
# Filter rows where 'AirLeak' is equal to 1
air_leak_df = df[df['Airleak'] == 1]

air_leak_df

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,Airleak
562564,2020-04-18 00:00:01,-0.018,8.248,8.238,-0.024,8.248,49.450,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1
562565,2020-04-18 00:00:13,-0.018,8.248,8.238,-0.024,8.248,49.450,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1
562566,2020-04-18 00:00:24,-0.018,8.248,8.238,-0.024,8.248,49.450,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1
562567,2020-04-18 00:00:36,-0.018,8.248,8.238,-0.024,8.248,49.450,0.0400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
562568,2020-04-18 00:00:49,-0.018,8.248,8.238,-0.024,8.248,49.450,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1172710,2020-07-15 18:59:20,-0.004,8.846,8.832,-0.008,8.848,73.650,3.5600,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1
1172711,2020-07-15 18:59:30,-0.006,8.816,8.800,-0.010,8.818,73.600,3.5575,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1
1172712,2020-07-15 18:59:40,-0.004,8.784,8.770,-0.008,8.788,73.500,3.6325,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1
1172713,2020-07-15 18:59:50,-0.006,8.754,8.738,-0.008,8.758,73.325,3.5475,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1


## KNN TRIAL

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Drop the 'timestamp' column
df2 = df2.drop('timestamp', axis=1)

# Standardize the data
scaler = StandardScaler()
df2_standardized = scaler.fit_transform(df2)

df2.head()

,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,Airleak
0,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
1,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
2,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
3,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
4,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0


In [ ]:
# Apply K-means clustering
kmeans = KMeans(n_clusters=2, random_state=42)
df2['Cluster'] = kmeans.fit_predict(df2_standardized)


df2.head()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,Airleak,Cluster
0,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,0
1,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,0
2,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,0
3,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,0
4,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,0


In [ ]:
# Identify potential air leaks based on cluster assignment

air_leak_threshold = 1
df2['Airleak'] = (df2['Cluster'] == air_leak_threshold).astype(int)

# Drop the 'Cluster' column
#df2 = df2.drop('Cluster', axis=1)

df2.head()

,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,Airleak,Cluster
0,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,0
1,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,0
2,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,0
3,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,0
4,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,0


In [ ]:
result_df = df2[df2['Airleak'] == 1]
result_df

,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,Airleak,Cluster
140,-0.014,8.068,7.804,-0.024,8.072,49.825,4.3150,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1,1
141,8.436,8.086,-0.028,-0.022,8.082,49.850,5.8400,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1,1
142,8.726,8.380,-0.024,-0.024,8.382,49.675,5.9725,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1,1
143,8.978,8.642,-0.020,-0.024,8.640,49.675,6.0500,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1,1
144,9.218,8.820,-0.020,-0.024,8.816,49.625,6.1025,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516867,9.890,9.486,-0.014,-0.022,9.482,59.075,6.0800,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1,1
1516868,8.166,9.648,-0.014,-0.022,9.646,59.775,5.7175,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1,1
1516869,10.162,9.760,-0.014,-0.022,9.756,60.850,6.1550,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1,1
1516870,10.332,9.942,-0.014,-0.022,9.940,61.675,6.1575,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1,1
